In [0]:
import dlt
from pyspark.sql.functions import col


# Bronze Silver Gold Layers for Master Data

In [0]:
@dlt.table(
    comment="Bronze layer - raw master customer data"
)
def bronze_customer():
    return spark.read.table("sachin_dlt_jun282025.master_data")


In [0]:
@dlt.table(
    comment="Silver layer - filtered active customers"
)
def silver_customer():
    df = dlt.read("bronze_customer")
    return df.filter(col("gender") == "Male")


In [0]:
@dlt.table(
    comment="Gold layer - customer records with gender flag"
)
def gold_customer_summary():
    df = dlt.read("silver_customer")
    return df.withColumn("is_female", col("gender") == "Female")


# Bronze Silver Gold Layers for Support Data

In [0]:
@dlt.table(
    comment="Bronze layer - raw support customer data"
)
def support_customer():
    return spark.read.table("sachin_dlt_jun282025.support_data")
    

In [0]:
@dlt.table(
    comment="Silver layer - filtered support active customers"
)
def silver_support_customer():
    df = dlt.read("support_customer")
    return df.filter(col("CVV_No") <= 1500)

In [0]:
from pyspark.sql.functions import current_timestamp

@dlt.table(
    comment="Gold layer - support data with processing timestamp"
)
def gold_support_summary():
    df = dlt.read("silver_support_customer")
    return df.withColumn("ingestion_time", current_timestamp())


In [0]:
from pyspark.sql.functions import col, current_timestamp
import dlt

@dlt.table(
    comment="Gold layer - unified customer and support data"
)
def gold_combined_summary():
    # Read both gold-level input tables
    customer_df = dlt.read("gold_customer_summary")
    support_df = dlt.read("gold_support_summary")

    # Perform an inner join on CustomerID
    joined_df = customer_df.join(support_df, on="CustomerID", how="inner")

    return joined_df
